In [3]:
from pyspark.sql import (
     SparkSession,
     functions as F,
     types)
spark=SparkSession.builder.appName("spark").getOrCreate()

In [29]:
df_pyspark=spark.read.parquet('orders_data.parquet')
df_pyspark.show(5)

+-------------------+--------+--------------------+-----------------+------------+--------------------+----------------+----------+----------+--------+-------+
|         order_date|order_id|             product|       product_id|    category|    purchase_address|quantity_ordered|price_each|cost_price|turnover| margin|
+-------------------+--------+--------------------+-----------------+------------+--------------------+----------------+----------+----------+--------+-------+
|2023-01-22 21:25:00|  141234|              iPhone|5.638008983335E12|   Vêtements|944 Walnut St, Bo...|               1|     700.0|     231.0|   700.0|  469.0|
|2023-01-28 14:15:00|  141235|Lightning Chargin...|5.563319511488E12|Alimentation|185 Maple St, Por...|               1|     14.95|     7.475|   14.95|  7.475|
|2023-01-17 13:33:00|  141236|    Wired Headphones| 2.11397339522E12|   Vêtements|538 Adams St, San...|               2|     11.99|     5.995|   23.98|  11.99|
|2023-01-05 20:33:00|  141237|    27in F

In [5]:
df_pyspark.printSchema()

root
 |-- order_date: timestamp_ntz (nullable = true)
 |-- order_id: long (nullable = true)
 |-- product: string (nullable = true)
 |-- product_id: double (nullable = true)
 |-- category: string (nullable = true)
 |-- purchase_address: string (nullable = true)
 |-- quantity_ordered: long (nullable = true)
 |-- price_each: double (nullable = true)
 |-- cost_price: double (nullable = true)
 |-- turnover: double (nullable = true)
 |-- margin: double (nullable = true)



In [9]:
df_pyspark=(
    df_pyspark.withColumn('time_of_day',
                          F.when((F.hour('order_date')>=0) & (F.hour('order_date')<=5),'night')
                          .when((F.hour('order_date')>=6) & (F.hour('order_date')<=13),'morning')
                          .when((F.hour('order_date')>=14) & (F.hour('order_date')<=19),'afternoon')
                          .when((F.hour('order_date')>=20) & (F.hour('order_date')<=23),'evening')
                          .otherwise(None)
)
.filter(F.col('time_of_day') !='night')
.withColumn('order_date',F.col('order_date').cast(types.DateType()))
)

In [10]:
df_pyspark.show(5)

+----------+--------+--------------------+-----------------+------------+--------------------+----------------+----------+----------+--------+-------+-----------+
|order_date|order_id|             product|       product_id|    category|    purchase_address|quantity_ordered|price_each|cost_price|turnover| margin|time_of_day|
+----------+--------+--------------------+-----------------+------------+--------------------+----------------+----------+----------+--------+-------+-----------+
|2023-01-22|  141234|              iPhone|5.638008983335E12|   Vêtements|944 Walnut St, Bo...|               1|     700.0|     231.0|   700.0|  469.0|    evening|
|2023-01-28|  141235|Lightning Chargin...|5.563319511488E12|Alimentation|185 Maple St, Por...|               1|     14.95|     7.475|   14.95|  7.475|  afternoon|
|2023-01-17|  141236|    Wired Headphones| 2.11397339522E12|   Vêtements|538 Adams St, San...|               2|     11.99|     5.995|   23.98|  11.99|    morning|
|2023-01-05|  141237| 

In [11]:
df_pyspark=(
    df_pyspark.withColumn('product', F.lower('product'))
    .withColumn('category', F.lower('category'))
    .filter(F.col('product').contains('tv'))

)

In [12]:
df_pyspark.show(5)

+----------+--------+-------------+-----------------+------------+--------------------+----------------+----------+----------+--------+------+-----------+
|order_date|order_id|      product|       product_id|    category|    purchase_address|quantity_ordered|price_each|cost_price|turnover|margin|time_of_day|
+----------+--------+-------------+-----------------+------------+--------------------+----------------+----------+----------+--------+------+-----------+
|2023-01-03|  141248|flatscreen tv| 4.06275646306E12|électronique|363 Spruce St, Au...|               1|     300.0|      99.0|   300.0| 201.0|    evening|
|2023-01-02|  141283|flatscreen tv|8.633300480139E12|électronique|68 Hickory St, Se...|               1|     300.0|      99.0|   300.0| 201.0|  afternoon|
|2023-01-09|  141331|flatscreen tv|7.572900898157E12|électronique|299 Park St, San ...|               1|     300.0|      99.0|   300.0| 201.0|  afternoon|
|2023-01-17|  141366|flatscreen tv|9.422250408078E12|      sports|803 

In [24]:
df_pyspark=(
    df_pyspark.withColumn('adress_purchase',F.split(F.col('purchase_address'),' '))
    .withColumn('purchase_state',F.col("adress_purchase").getItem(F.size('adress_purchase')-2))
    .drop('address_purchase')
)

In [19]:
df_pyspark.show(5)

+----------+--------+-------------+-----------------+------------+--------------------+----------------+----------+----------+--------+------+-----------+--------------+
|order_date|order_id|      product|       product_id|    category|    purchase_address|quantity_ordered|price_each|cost_price|turnover|margin|time_of_day|purchase_state|
+----------+--------+-------------+-----------------+------------+--------------------+----------------+----------+----------+--------+------+-----------+--------------+
|2023-01-03|  141248|flatscreen tv| 4.06275646306E12|électronique|363 Spruce St, Au...|               1|     300.0|      99.0|   300.0| 201.0|    evening|          NULL|
|2023-01-02|  141283|flatscreen tv|8.633300480139E12|électronique|68 Hickory St, Se...|               1|     300.0|      99.0|   300.0| 201.0|  afternoon|          NULL|
|2023-01-09|  141331|flatscreen tv|7.572900898157E12|électronique|299 Park St, San ...|               1|     300.0|      99.0|   300.0| 201.0|  aftern

In [26]:
n_states=df_pyspark.select('purchase_state').distinct().count()
print(f"Number of states: {n_states}")

Number of states: 8


In [31]:
avg = df_pyspark.groupby('category').agg(F.avg('cost_price')).alias('avg_cost_price')
avg.show()

+------------+-----------------+
|    category|  avg(cost_price)|
+------------+-----------------+
|Électronique|69.73946748133805|
|      Sports| 69.5346502205941|
|   Vêtements|69.47677533673904|
|Alimentation|69.92548114240572|
+------------+-----------------+

